<a href="https://colab.research.google.com/github/humazafar2703/Assignment3_Python/blob/main/Copy_of_Rainfall_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Step 1: Install required packages
!pip install kagglehub geopy --no-cache-dir

# Step 2: Import libraries
import kagglehub
import os
import zipfile
import pandas as pd
from geopy.geocoders import Nominatim
import time


In [3]:
# Step 3: Download dataset using KaggleHub
uk_rainfall = kagglehub.dataset_download("jakewright/2m-daily-weather-history-uk")
print("✅ Dataset downloaded to:", uk_rainfall)

✅ Dataset downloaded to: /kaggle/input/2m-daily-weather-history-uk


In [4]:
# Step 4: Check for ZIP file and extract it
zip_files = [f for f in os.listdir(uk_rainfall) if f.endswith('.zip')]
if zip_files:
    zip_path = os.path.join(uk_rainfall, zip_files[0])
    print("📦 Found ZIP file:", zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(uk_rainfall)
    print("✅ ZIP file extracted.")

In [5]:
# Step 5: Find CSV file
csv_file_path = None
for root, dirs, files in os.walk(uk_rainfall):
    for file in files:
        if file.endswith('.csv'):
            csv_file_path = os.path.join(root, file)
            break

if not csv_file_path:
    raise FileNotFoundError("❌ CSV file not found in the dataset.")

print("📄 Using CSV file:", csv_file_path)

📄 Using CSV file: /kaggle/input/2m-daily-weather-history-uk/all_weather_data.csv


In [6]:
# Step 6: Load CSV
df = pd.read_csv(csv_file_path)


In [7]:
# Step 7: Handle dates
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

start_date = df['date'].min()
end_date = df['date'].max()

# Step 8: Output date range
print(f"📅 Data covers from {start_date.date()} to {end_date.date()}")

📅 Data covers from 2009-01-01 to 2024-11-12


In [8]:
 #Now check unique locations
num_unique_locations = len(df['location'].unique())
print(f"Number of unique locations: {num_unique_locations}")

Number of unique locations: 504


In [10]:
!pip install geopy


In [9]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="uk-weather-study")
location_coords = []

for loc in df['location'].dropna().unique():
    try:
        location = geolocator.geocode(loc + ", UK")
        if location:
            location_coords.append({
                'location': loc,
                'latitude': location.latitude,
                'longitude': location.longitude
            })
        else:
            location_coords.append({'location': loc, 'latitude': None, 'longitude': None})
    except Exception as e:
        print(f"Error for {loc}: {e}")
    time.sleep(1)  # To avoid being blocked


In [12]:
# Assuming location_coords = {location: (lat, lon), ...}

# Create a DataFrame for locations and their coordinates
loc_coords_df = pd.DataFrame([
    {'location': loc, 'latitude': coords[0], 'longitude': coords[1]}
    for loc, coords in location_coords.items()
])

# Filter locations inside UK bounding box
uk_loc_df = loc_coords_df[
    (loc_coords_df['latitude'] >= 49.5) & (loc_coords_df['latitude'] <= 61.0) &
    (loc_coords_df['longitude'] >= -8.5) & (loc_coords_df['longitude'] <= 2.0)
]

uk_locations = uk_loc_df['location'].tolist()

print(f"Number of locations within UK bounds: {len(uk_locations)}")


Number of locations within UK bounds: 477


In [13]:
# Aggregate rainfall stats by location
rainfall_stats = df.groupby('location')['rain mm'].agg(['min', 'mean', 'max']).reset_index()
rainfall_stats.rename(columns={'min':'min_rain_mm', 'mean':'avg_rain_mm', 'max':'max_rain_mm'}, inplace=True)


In [14]:
rainfall_stats['latitude'] = rainfall_stats['location'].apply(lambda loc: location_coords.get(loc, (None, None))[0])
rainfall_stats['longitude'] = rainfall_stats['location'].apply(lambda loc: location_coords.get(loc, (None, None))[1])

# Drop rows without coordinates
rainfall_stats = rainfall_stats.dropna(subset=['latitude', 'longitude'])


In [29]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- STEP 1: Prepare Data ---

# Add latitude and longitude from the location_coords dictionary
rainfall_stats['latitude'] = rainfall_stats['location'].map(
    lambda loc: location_coords.get(loc, (None, None))[0]
)
rainfall_stats['longitude'] = rainfall_stats['location'].map(
    lambda loc: location_coords.get(loc, (None, None))[1]
)

# Keep only rows with valid coordinates and average rainfall
rainfall_stats = rainfall_stats.dropna(subset=['latitude', 'longitude', 'avg_rain_mm'])

# Extract arrays of lat, lon, and rainfall
lats = rainfall_stats['latitude'].values
lons = rainfall_stats['longitude'].values
rains = rainfall_stats['avg_rain_mm'].astype(float).values

# --- STEP 2: Create the Heatmap Layer ---

# Use Densitymapbox for heatmap-style shading
heatmap = go.Densitymapbox(
    lat=lats,
    lon=lons,
    z=rains,               # Use average rainfall to control intensity
    radius=30,             # Controls smoothness (bigger = smoother heat)
    colorscale='Blues',    # Color gradient for rainfall
    hovertemplate='Rainfall: %{z:.2f} mm<extra></extra>'
)

# --- STEP 3: Display Map ---

fig = go.Figure(heatmap)

fig.update_layout(
    mapbox=dict(
        style='carto-positron',            # Light map style
        center=dict(lat=54.5, lon=-3),     # Centered over the UK
        zoom=5.5                           # Good zoom level for UK
    ),
    title="UK Rainfall Heatmap (Average Rainfall in mm)",
    margin=dict(r=0, t=50, l=0, b=0),
    height=600
)

fig.show()


In [16]:
# Step 1: Extract year from date
df['year'] = df['date'].dt.year

# Step 2: Sum rainfall by location and year
yearly_rainfall = df.groupby(['location', 'year'])['rain mm'].sum().reset_index()

# Step 3: Find year with max rainfall per location
max_rain_year = yearly_rainfall.loc[yearly_rainfall.groupby('location')['rain mm'].idxmax()]
max_rain_year.rename(columns={'year': 'year_max_rain', 'rain mm': 'max_rainfall_mm'}, inplace=True)

# Step 4: Find year with min rainfall per location
min_rain_year = yearly_rainfall.loc[yearly_rainfall.groupby('location')['rain mm'].idxmin()]
min_rain_year.rename(columns={'year': 'year_min_rain', 'rain mm': 'min_rainfall_mm'}, inplace=True)

# Step 5: Merge max and min tables
rainfall_years = pd.merge(max_rain_year[['location', 'year_max_rain', 'max_rainfall_mm']],
                          min_rain_year[['location', 'year_min_rain', 'min_rainfall_mm']],
                          on='location')

# Display a sample
print(rainfall_years.head())


     location  year_max_rain  max_rainfall_mm  year_min_rain  min_rainfall_mm
0  Abengourou           2023           2274.6           2024            596.3
1   Amsterdam           2023           1165.8           2024            484.4
2     Ardkeen           2023            226.6           2024             70.6
3   Ardmillan           2023           1390.5           2024            444.7
4      Athens           2019            659.2           2024            124.7


In [17]:
avg_rainfall = df.groupby('location')['rain mm'].mean().reset_index()
avg_rainfall.rename(columns={'rain mm': 'avg_rain_mm'}, inplace=True)


In [18]:
top_wettest = avg_rainfall.sort_values(by='avg_rain_mm', ascending=False).head(10)
print("Top 10 Wettest Places (by average rainfall):")
print(top_wettest)


Top 10 Wettest Places (by average rainfall):
        location  avg_rain_mm
39      Bridgend     4.750727
482    Waterside     4.281071
13       Balmore     3.945787
92      Fluchter     3.945787
43       Buchley     3.945787
23      Bearsden     3.945787
18      Bardowie     3.945787
37   Blairskaith     3.945787
178    Milngavie     3.945787
179       Milton     3.945087


In [19]:
driest = avg_rainfall.sort_values(by='avg_rain_mm', ascending=True).head(10)
print("Top 10 Driest Places (by average rainfall):")
print(driest)


Top 10 Driest Places (by average rainfall):
         location  avg_rain_mm
158        Madrid     1.158914
4          Athens     1.338052
142        Lisbon     1.586926
447     Stockholm     1.645042
17      Barcelona     1.730903
30         Berlin     1.775430
45       Budapest     1.782437
291  Page's Green     1.810624
297      Palgrave     1.810624
259        Occold     1.810624


In [20]:
import plotly.graph_objects as go

def show_table(df, title):
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),
                    fill_color='lightblue',
                    align='left'),
        cells=dict(values=[df[col] for col in df.columns],
                   fill_color='white',
                   align='left'))
    ])
    fig.update_layout(title=title)
    fig.show()

show_table(top_wettest, "Top 10 Wettest Places in UK")
show_table(driest, "Top 10 Driest Places in UK")


In [21]:
# MODELLING PHASES

# Ensure 'date' is datetime type
df['date'] = pd.to_datetime(df['date'])

# Set date as index temporarily for resampling
df.set_index('date', inplace=True)

# Aggregate weekly rainfall sums per location
df_weekly = df.groupby('location').resample('W')['rain mm'].sum().reset_index()

print(df_weekly.head())


     location       date  rain mm
0  Abengourou 2009-01-04      0.8
1  Abengourou 2009-01-11      2.9
2  Abengourou 2009-01-18      2.0
3  Abengourou 2009-01-25      0.0
4  Abengourou 2009-02-01      1.5


In [22]:
#  2. Time-based Split into Train / Validation / Test
# We split based on weeks (after aggregation), preserving time order to avoid data leakage.

# Sort by date
df_weekly = df_weekly.sort_values('date')

# Get unique sorted weeks
weeks = df_weekly['date'].sort_values().unique()

# Define split indices
train_end = int(len(weeks) * 0.7)
val_end = int(len(weeks) * 0.85)

train_weeks = weeks[:train_end]
val_weeks = weeks[train_end:val_end]
test_weeks = weeks[val_end:]

# Split dataframes by weeks
train_df = df_weekly[df_weekly['date'].isin(train_weeks)]
val_df = df_weekly[df_weekly['date'].isin(val_weeks)]
test_df = df_weekly[df_weekly['date'].isin(test_weeks)]

print(f"Train weeks: {len(train_weeks)}")
print(f"Validation weeks: {len(val_weeks)}")
print(f"Test weeks: {len(test_weeks)}")

print("Train sample:")
print(train_df.head())



Train weeks: 580
Validation weeks: 124
Test weeks: 125
Train sample:
          location       date  rain mm
0       Abengourou 2009-01-04      0.8
250959     Parkway 2009-01-04      0.0
250152    Parkside 2009-01-04      0.0
249345    Parkgate 2009-01-04      0.0
248538   Park Lane 2009-01-04      0.0


In [111]:
print(f"Train weeks: {len(train_weeks)}")
print(f"Validation weeks: {len(val_weeks)}")
print(f"Test weeks: {len(test_weeks)}")


Train weeks: 580
Validation weeks: 124
Test weeks: 125


In [32]:
# Install PyTorch Geometric and its dependencies (CPU version)
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric \
  -f https://data.pyg.org/whl/torch-2.0.0+cpu.html


Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html


In [33]:
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data as GeoData


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning:

An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning:

An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:113: UserWarning:

An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning:

An issue occurred while importing 'torc

In [34]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data as GeoData
import tensorflow as tf
from tensorflow.keras import layers

In [35]:
#Step 1: Prepare sequences and datasets
SEQ_LEN = 4
PRED_LEN = 1

def create_sequences(df):
    """
    Create (X,y) for each location: X=last SEQ_LEN rain, y=next PRED_LEN rain
    Returns list of dicts with keys: location, X, y, date
    """
    sequences = []
    for loc in df['location'].unique():
        loc_data = df[df['location'] == loc].sort_values('date')
        rain = loc_data['rain mm'].values
        dates = loc_data['date'].values
        for i in range(len(rain) - SEQ_LEN - PRED_LEN + 1):
            X = rain[i:i+SEQ_LEN]
            y = rain[i+SEQ_LEN:i+SEQ_LEN+PRED_LEN]
            sequences.append({'location': loc, 'X': X, 'y': y, 'date': dates[i+SEQ_LEN+PRED_LEN-1]})
    return sequences

train_seq = create_sequences(train_df)
val_seq = create_sequences(val_df)
test_seq = create_sequences(test_df)

print(f"Train sequences: {len(train_seq)}, Val sequences: {len(val_seq)}, Test sequences: {len(test_seq)}")



Train sequences: 290263, Val sequences: 60480, Test sequences: 49919


In [47]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

latent_dim = 8
input_dim = SEQ_LEN  # e.g. 4
output_dim = PRED_LEN  # e.g. 1

def build_generator():
    input_seq = layers.Input(shape=(input_dim,))
    noise = layers.Input(shape=(latent_dim,))
    x = layers.Concatenate()([input_seq, noise])
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(output_dim)(x)
    return tf.keras.Model([input_seq, noise], x)

def build_discriminator():
    pred = layers.Input(shape=(output_dim,))
    x = layers.Dense(64, activation='relu')(pred)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(pred, x)

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

discriminator.trainable = False
input_seq = layers.Input(shape=(input_dim,))
noise = layers.Input(shape=(latent_dim,))
generated = generator([input_seq, noise])
validity = discriminator(generated)
gan = tf.keras.Model([input_seq, noise], validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')


def train_gan(train_dataset, epochs=1000, batch_size=32):
    for epoch in range(epochs):
        idx = np.random.randint(0, len(train_dataset), batch_size)
        real_X = np.array([train_dataset[i][0].numpy() for i in idx])
        real_y = np.array([train_dataset[i][1].numpy() for i in idx])

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_y = generator.predict([real_X, noise], verbose=0)

        d_loss_real = discriminator.train_on_batch(real_y, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_y, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch([real_X, noise], np.ones((batch_size, 1)))

        if epoch % 100 == 0:
            print(f"Epoch {epoch} - D loss: {d_loss:.4f} - G loss: {g_loss:.4f}")


In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RainFormer(nn.Module):
    def __init__(self, seq_len=SEQ_LEN, pred_len=PRED_LEN, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.d_model = d_model

        # Input embedding: project scalar rainfall to d_model dims
        self.input_proj = nn.Linear(1, d_model)

        # Positional encoding
        self.pos_embedding = nn.Parameter(torch.randn(seq_len, d_model))

        # Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Output layer projects last encoded token to predicted rainfall(s)
        self.fc_out = nn.Linear(d_model, pred_len)

    def forward(self, x):
        """
        x shape: (batch_size, seq_len)
        """
        batch_size = x.size(0)

        # Reshape to (batch_size, seq_len, 1) and embed
        x = x.unsqueeze(-1)  # (B, seq_len, 1)
        x = self.input_proj(x)  # (B, seq_len, d_model)

        # Add positional embedding
        x = x + self.pos_embedding.unsqueeze(0)  # broadcast to batch

        # Transformer expects shape (seq_len, batch_size, d_model)
        x = x.permute(1, 0, 2)

        x = self.transformer_encoder(x)  # (seq_len, batch, d_model)

        # Use last time step's output for prediction
        x_last = x[-1, :, :]  # (batch, d_model)

        out = self.fc_out(x_last)  # (batch, pred_len)

        return out


In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GraphCastModel(nn.Module):
    def __init__(self, in_channels=SEQ_LEN, hidden_channels=32, out_channels=PRED_LEN):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        """
        x: node features tensor, shape [num_nodes, in_channels]
        edge_index: tensor defining graph connectivity, shape [2, num_edges]
        """
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x  # shape: [num_nodes, out_channels]


In [54]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
import numpy as np

def compute_metrics(trues, preds, name='Model'):
    rmse = mean_squared_error(trues, preds, squared=False)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    corr, _ = pearsonr(trues, preds)

    true_rain = (trues > 0.1).astype(int)
    pred_rain = (preds > 0.1).astype(int)
    binary_accuracy = np.mean(true_rain == pred_rain)

    print(f"\n📊 Evaluation of {name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  Pearson Correlation: {corr:.4f}")
    print(f"  Rain/No-Rain Accuracy: {binary_accuracy:.4f}")

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "corr": corr,
        "binary_accuracy": binary_accuracy
    }


In [55]:
def evaluate_model_tf(generator, dataset, latent_dim=8, batch_size=32, name='Model'):
    preds, trues = [], []
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i+batch_size]
        X_batch = np.array([x for x, y in batch])
        y_true = np.array([y for x, y in batch]).flatten()

        noise = np.random.normal(0, 1, (len(X_batch), latent_dim))
        preds_batch = generator.predict([X_batch, noise]).flatten()

        preds.extend(preds_batch)
        trues.extend(y_true)

    preds = np.array(preds)
    trues = np.array(trues)
    return compute_metrics(trues, preds, name)


In [59]:
def evaluate_model_pytorch(model, loader, edge_index, name='Model'):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            # X_batch shape: [batch_size, seq_len], permute to [seq_len, batch_size, 1]
            X_batch = X_batch.unsqueeze(-1).permute(1, 0, 2)
            y_true = y_batch.numpy().flatten()

            output = model(X_batch, edge_index).squeeze().cpu().numpy().flatten()
            preds.extend(output)
            trues.extend(y_true)

    preds = np.array(preds)
    trues = np.array(trues)

    return compute_metrics(trues, preds, name)


In [83]:
from torch_geometric.nn import TransformerConv

class RainFormerModel(torch.nn.Module):
    def __init__(self, in_channels=SEQ_LEN, hidden_channels=32, out_channels=PRED_LEN, heads=4):
        super().__init__()
        self.conv1 = TransformerConv(in_channels, hidden_channels, heads=heads, dropout=0.1)
        self.conv2 = TransformerConv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

rainformer_model = RainFormerModel()


In [84]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from scipy.stats import pearsonr
import torch

def compute_metrics(trues, preds, name='Model'):
    rmse = np.sqrt(mean_squared_error(trues, preds))  # fixed RMSE calculation
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    corr, _ = pearsonr(trues, preds)

    true_rain = (trues > 0.1).astype(int)
    pred_rain = (preds > 0.1).astype(int)
    binary_accuracy = np.mean(true_rain == pred_rain)

    print(f"\n📊 Evaluation of {name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  Pearson Correlation: {corr:.4f}")
    print(f"  Rain/No-Rain Accuracy: {binary_accuracy:.4f}")

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "corr": corr,
        "binary_accuracy": binary_accuracy
    }

# Evaluate TensorFlow GenCast model
def evaluate_model_tf(generator, dataset, latent_dim, batch_size=32, name='GenCast'):
    preds, trues = [], []
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i+batch_size]
        real_X = np.array([x[0].numpy() for x in batch])
        real_y = np.array([x[1].numpy() for x in batch])

        noise = np.random.normal(0, 1, (len(batch), latent_dim))
        fake_y = generator.predict([real_X, noise], verbose=0)

        preds.extend(fake_y.flatten())
        trues.extend(real_y.flatten())

    preds = np.array(preds)
    trues = np.array(trues)

    return compute_metrics(trues, preds, name)

# Evaluate PyTorch models (GraphCast, RainFormer)
def evaluate_model_pytorch(model, loader, edge_index=None, name='Model'):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            # X_batch shape: (batch_size, SEQ_LEN)
            # Adjust dims for graph models if needed
            if edge_index is not None:
                # GraphCast/RainFormer expects x and edge_index
                out = model(X_batch, edge_index)
                output = out.squeeze().cpu().numpy()
            else:
                output = model(X_batch).squeeze().cpu().numpy()

            y_true = y_batch.numpy().flatten()
            preds.extend(output)
            trues.extend(y_true)

    preds = np.array(preds)
    trues = np.array(trues)

    return compute_metrics(trues, preds, name)



In [85]:
def evaluate_model_tf(generator, sequences, latent_dim=8, batch_size=32, name='GenCast'):
    preds = []
    trues = []

    # Process sequences in batches
    for i in range(0, len(sequences), batch_size):
        batch = sequences[i:i+batch_size]
        X_batch = np.array([seq['X'] for seq in batch])
        y_true = np.array([seq['y'] for seq in batch]).flatten()

        noise = np.random.normal(0, 1, (len(batch), latent_dim))
        y_pred = generator.predict([X_batch, noise], verbose=0).flatten()

        preds.extend(y_pred)
        trues.extend(y_true)

    preds = np.array(preds)
    trues = np.array(trues)

    # Metrics
    rmse = mean_squared_error(trues, preds, squared=False)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    corr, _ = pearsonr(trues, preds)
    true_rain = (trues > 0.1).astype(int)
    pred_rain = (preds > 0.1).astype(int)
    binary_accuracy = np.mean(true_rain == pred_rain)

    print(f"\n📊 Evaluation of {name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  Pearson Correlation: {corr:.4f}")
    print(f"  Rain/No-Rain Accuracy: {binary_accuracy:.4f}")

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "corr": corr,
        "binary_accuracy": binary_accuracy
    }


In [75]:
def evaluate_model_tf(model, sequences, latent_dim, batch_size=32, name='Model'):
    preds, trues = [], []

    for i in range(0, len(sequences), batch_size):
        batch = sequences[i:i+batch_size]
        X_batch = np.array([seq['X'] if isinstance(seq, dict) else seq[0] for seq in batch])
        y_true = np.array([seq['y'] if isinstance(seq, dict) else seq[1] for seq in batch])

        noise = np.random.normal(0, 1, (len(X_batch), latent_dim))
        y_pred = model.predict([X_batch, noise])

        preds.extend(y_pred.flatten())
        trues.extend(y_true.flatten())

    preds = np.array(preds)
    trues = np.array(trues)

    rmse = np.sqrt(mean_squared_error(trues, preds))  # Fix here: compute RMSE manually
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    corr, _ = pearsonr(trues, preds)

    true_rain = (trues > 0.1).astype(int)
    pred_rain = (preds > 0.1).astype(int)
    binary_accuracy = np.mean(true_rain == pred_rain)

    print(f"\n📊 Evaluation of {name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  Pearson Correlation: {corr:.4f}")
    print(f"  Rain/No-Rain Accuracy: {binary_accuracy:.4f}")

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "corr": corr,
        "binary_accuracy": binary_accuracy
    }


In [76]:
small_test_seq = test_seq[:250]

gen_cast_metrics = evaluate_model_tf(generator, small_test_seq, latent_dim=latent_dim, batch_size=32, name='GenCast')
print("GenCast evaluation done. Metrics:", gen_cast_metrics)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

📊 Evaluation of GenCast:
  RMSE: 28.9504
  MAE:  22.4828
  R²:   -1.5079
  Pearson Correlation: 0.0904
  Rain/No-Rain Accuracy: 0.4560
GenCast evaluation done. Metrics: {'rmse': 28.950373938600613, 'mae': 22.482812502838673, 'r2': -1.507923861832448, 'corr': 0.09035792375270127, 'binary_accuracy': 0.456}


In [86]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
import torch

def evaluate_model_pytorch(model, loader, edge_index, device, name='Model'):
    model.eval()
    preds, trues = [], []

    edge_index = edge_index.to(device)

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            output = model(X_batch, edge_index)
            preds.append(output.cpu().numpy().flatten())
            trues.append(y_batch.cpu().numpy().flatten())

    preds = np.concatenate(preds)
    trues = np.concatenate(trues)

    mse = mean_squared_error(trues, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    corr, _ = pearsonr(trues, preds)

    true_rain = (trues > 0.1).astype(int)
    pred_rain = (preds > 0.1).astype(int)
    binary_accuracy = np.mean(true_rain == pred_rain)

    print(f"\n{name} Evaluation:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R²: {r2:.4f}")
    print(f"  Pearson Correlation: {corr:.4f}")
    print(f"  Rain/No-Rain Accuracy: {binary_accuracy:.4f}")

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "corr": corr,
        "binary_accuracy": binary_accuracy
    }

# --- Usage ---

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move models and data to device
graphcast_model.to(device)
rainformer_model.to(device)
edge_index = edge_index.to(device)

# Evaluate GraphCast
graphcast_metrics = evaluate_model_pytorch(graphcast_model, test_loader, edge_index, device, name='GraphCast')

# Evaluate RainFormer
rainformer_metrics = evaluate_model_pytorch(rainformer_model, test_loader, edge_index, device, name='RainFormer')

# Print comparison
print("\n=== Model Performance Comparison ===")
print("GraphCast Metrics:", graphcast_metrics)
print("RainFormer Metrics:", rainformer_metrics)



GraphCast Evaluation:
  RMSE: 29.4349
  MAE: 22.1446
  R²: -1.2916
  Pearson Correlation: 0.1400
  Rain/No-Rain Accuracy: 0.2361

RainFormer Evaluation:
  RMSE: 29.5121
  MAE: 22.1832
  R²: -1.3036
  Pearson Correlation: -0.1077
  Rain/No-Rain Accuracy: 0.0498

=== Model Performance Comparison ===
GraphCast Metrics: {'rmse': 29.43491334342869, 'mae': 22.144628524780273, 'r2': -1.2915856838226318, 'corr': 0.14001912, 'binary_accuracy': 0.23610248602736433}
RainFormer Metrics: {'rmse': 29.51205973702742, 'mae': 22.18318748474121, 'r2': -1.3036136627197266, 'corr': -0.107736975, 'binary_accuracy': 0.04982070954947014}


In [91]:
import pandas as pd

data = {
    "Metric": ["RMSE", "MAE", "R² Score", "Pearson Correlation", "Rain/No-Rain Accuracy"],
    "GenCast": [
        gen_cast_metrics["rmse"],
        gen_cast_metrics["mae"],
        gen_cast_metrics["r2"],
        gen_cast_metrics["corr"],
        gen_cast_metrics["binary_accuracy"]
    ],
    "GraphCast": [
        graphcast_metrics["rmse"],
        graphcast_metrics["mae"],
        graphcast_metrics["r2"],
        graphcast_metrics["corr"],
        graphcast_metrics["binary_accuracy"]
    ],
    "RainFormer": [
        rainformer_metrics["rmse"],
        rainformer_metrics["mae"],
        rainformer_metrics["r2"],
        rainformer_metrics["corr"],
        rainformer_metrics["binary_accuracy"]
    ]
}

df = pd.DataFrame(data)
print(df.to_markdown(index=False))


| Metric                |    GenCast |   GraphCast |   RainFormer |
|:----------------------|-----------:|------------:|-------------:|
| RMSE                  | 28.9504    |   29.4349   |   29.5121    |
| MAE                   | 22.4828    |   22.1446   |   22.1832    |
| R² Score              | -1.50792   |   -1.29159  |   -1.30361   |
| Pearson Correlation   |  0.0903579 |    0.140019 |   -0.107737  |
| Rain/No-Rain Accuracy |  0.456     |    0.236102 |    0.0498207 |
